In [2]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName("UDF_example.com").getOrCreate()

24/09/11 15:31:39 WARN Utils: Your hostname, AI-CJB-LAP-459 resolves to a loopback address: 127.0.1.1; using 192.168.1.164 instead (on interface wlp0s20f3)
24/09/11 15:31:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/11 15:31:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [23]:
columns=["Seqno","Name"]
data=[("1","John"),
      ("2","smith"),
      ("3","amy")]


In [24]:
df=spark.createDataFrame(data=data,schema=columns)
df.show()

+-----+-----+
|Seqno| Name|
+-----+-----+
|    1| John|
|    2|smith|
|    3|  amy|
+-----+-----+



TO CREATE A PYTHON FUNCTION TO CONVERT THE CASE OF A STRING

In [25]:
def convertCase(str):
    res=""
    arr=str.split(" ")
    for x in arr:
        res=res+x[0:1].upper()+x[1:len(x)]+" "
    return res

CONVERT THE PYTHON FUCTION TO THE PYSPARK-UDF

In [26]:
from pyspark.sql.functions import col,udf
from pyspark.sql.types import StringType

In [18]:
convertUDF=udf(lambda z: convertCase(z))

USE UDF WITH DATAFRAME
Use UDF with pyspark dataframe select()

In [27]:

df.select(col("Seqno"), \
    convertUDF(col("Name")).alias("Name") ) \
   .show(truncate=False)

+-----+------+
|Seqno|Name  |
+-----+------+
|1    |John  |
|2    |Smith |
|3    |Amy   |
+-----+------+



USING UDF WITH PYSPARK DATAFRAME WITHCOLUMN

In [29]:
def upperCase(str):
    return str.upper()

In [30]:
ucaseUDF=udf(lambda x:upperCase(x),StringType())

In [33]:
df.withColumn("cureated_name",ucaseUDF(col("Name")))\
    .show()
    

df.write.option("header",True) \
        .partitionBy("Name") \
        .mode("overwrite") \
        .csv("Names")

+-----+-----+-------------+
|Seqno| Name|cureated_name|
+-----+-----+-------------+
|    1| John|         JOHN|
|    2|smith|        SMITH|
|    3|  amy|          AMY|
+-----+-----+-------------+

